<a href="https://colab.research.google.com/github/shailymishra/Game-Theory/blob/master/Game_Theory_Fairness_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools
from ortools.linear_solver import pywraplp
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def calculateMMSvalue(valuation):
  solver = pywraplp.Solver.CreateSolver('SCIP')
  infinity = solver.infinity()

  x = {}
  for i in range(num_agents):
    for k in range(num_items):
      x[i,k] =  x1 = solver.IntVar(0.0, infinity, 'x[%i,%i]' %(i,k))
      ## allocation can be either 0 or 1
      solver.Add(x[i,k] <= 1)

  y = solver.NumVar(0,infinity,'y')

  print('Number of variables =', solver.NumVariables())
  print(solver)

  ## eack item should be only allocated once
  for k in range(num_items):
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) <=1)
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) >=1)
  
  ## Complete allocation 
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) <= num_items)
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) >= num_items)
  
  ## Take the minimum bundle valuation
  for i in range(num_agents):
    solver.Add(solver.Sum(valuation[k]*x[i,k] for k in range(num_items)) >= y)
  
  print('Number of constraints =', solver.NumConstraints())
  ## Maximize the bundle
  solver.Maximize(y)
  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
      print('Solution:')
      print('Objective value MMS Share is =', solver.Objective().Value())
      for i in range(num_agents):
        print('______________________________',i,'______________________________')
        for k in range(num_items):
          print('x =', x[i,k].solution_value())
        print('________________________________________________')
  else:
      print('The problem does not have an optimal solution.')
  return solver.Objective().Value()

def findMMSAllocation(valuation_profile, mmsvalue=[]):

  for i in range(num_agents):
    mmsvalue.append(calculateMMSvalue(valuation_profile[i]))
  print(mmsvalue)

  solver = pywraplp.Solver.CreateSolver('SCIP')
  infinity = solver.infinity()

  x = {}
  for i in range(num_agents):
    for k in range(num_items):
      x[i,k] =  x1 = solver.IntVar(0.0, infinity, 'x[%i,%i]' %(i,k))
      ## allocation can be either 0 or 1
      solver.Add(x[i,k] <= 1)

  y = solver.NumVar(0,infinity,'y')

  print('Number of variables =', solver.NumVariables())
  print(solver)

  ## eack item should be only allocated once
  for k in range(num_items):
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) <=1)
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) >=1)
  
  ## Complete allocation 
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) <= num_items)
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) >= num_items)
  
  ## Each agent should get its mms value
  for i in range(num_agents):
    solver.Add(solver.Sum(valuation_profile[i][k]*x[i,k] for k in range(num_items)) >= mmsvalue[i])
  
  print('Number of constraints =', solver.NumConstraints())
  print(solver)
  ## Maximize 1
  solver.Maximize(1)
  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
      print('Solution:')
      print('Objective value MMS Share is =', solver.Objective().Value())
      for i in range(num_agents):
        print('______________________________',i,'______________________________')
        for k in range(num_items):
          print('x =', x[i,k].solution_value())
        print('________________________________________________')
  else:
      print('The problem does not have an optimal solution.')



def calculateEF(valuation_profile):
  solver = pywraplp.Solver.CreateSolver('SCIP')
  infinity = solver.infinity()

  x = {}
  for i in range(num_agents):
    for k in range(num_items):
      x[i,k] =  x1 = solver.IntVar(0.0, infinity, 'x[%i,%i]' %(i,k))
      ## allocation can be either 0 or 1
      solver.Add(x[i,k] <= 1)

  y = solver.NumVar(0,infinity,'y')

  print('Number of variables =', solver.NumVariables())
  print(solver)

  ## eack item should be only allocated once
  for k in range(num_items):
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) <=1)
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) >=1)
  
  ## Complete allocation 
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) <= num_items)
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) >= num_items)
  
  ## envyfreeness constraint
  for i in range(num_agents):
    for j in range(i+1,num_agents):
      print('j', j)
      ## i shouldnot envy j
      solver.Add(solver.Sum(valuation_profile[i,k]* x[i,k] for k in range(num_items)) >= solver.Sum(valuation_profile[i,k]* x[j,k] for k in range(num_items) ))
      ## j shouldnot envy i
      solver.Add(solver.Sum(valuation_profile[j,k]* x[j,k] for k in range(num_items)) >= solver.Sum(valuation_profile[j,k]* x[i,k] for k in range(num_items) ))

  
  print('Number of constraints =', solver.NumConstraints())
  print(solver.constraints())
  ## Maximize the bundle
  solver.Maximize(1)
  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
      print('Solution:')
      print('Objective value MMS Share is =', solver.Objective().Value())
      for i in range(num_agents):
        print('______________________________',i,'______________________________')
        for k in range(num_items):
          print('x =', x[i,k].solution_value())
        print('________________________________________________')
  else:
      print('The problem does not have an optimal solution.')
  return solver.Objective().Value()

def calculateEFwihUSW(valuation_profile):
  solver = pywraplp.Solver.CreateSolver('SCIP')
  infinity = solver.infinity()

  x = {}
  for i in range(num_agents):
    for k in range(num_items):
      x[i,k] =  x1 = solver.IntVar(0.0, infinity, 'x[%i,%i]' %(i,k))
      ## allocation can be either 0 or 1
      solver.Add(x[i,k] <= 1)

  y = solver.NumVar(0,infinity,'y')

  print('Number of variables =', solver.NumVariables())
  print(solver)

  ## eack item should be only allocated once
  for k in range(num_items):
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) <=1)
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) >=1)
  
  ## Complete allocation 
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) <= num_items)
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) >= num_items)
  
  ## envyfreeness constraint
  for i in range(num_agents):
    for j in range(i+1,num_agents):
      # print('j', j)
      ## i shouldnot envy j
      solver.Add(solver.Sum(valuation_profile[i,k]* x[i,k] for k in range(num_items)) >= solver.Sum(valuation_profile[i,k]* x[j,k] for k in range(num_items) ))
      ## j shouldnot envy i
      solver.Add(solver.Sum(valuation_profile[j,k]* x[j,k] for k in range(num_items)) >= solver.Sum(valuation_profile[j,k]* x[i,k] for k in range(num_items) ))

  
  print('Number of constraints =', solver.NumConstraints())
  print(solver.constraints())
  ## Maximize the bundle
  solver.Maximize(solver.Sum(valuation_profile[i,k]*x[i,k] for i in range(num_agents) for k in range(num_items)))
  start = time.time()
  print("hello")
  status = solver.Solve()
  end = time.time()

  allocation = []
  if status == pywraplp.Solver.OPTIMAL:
      print('Solution:')
      print('Objective value MMS Share is =', solver.Objective().Value())
      for i in range(num_agents):
        userallocation = []
        # print('______________________________',i,'______________________________')
        for k in range(num_items):
          # print('x =', x[i,k].solution_value())
          userallocation.append(x[i,k].solution_value())
        allocation.append(userallocation)
        # print('________________________________________________')
  else:
      print('The problem does not have an optimal solution.')
  print(allocation)
  allocation = np.array(allocation)
  print(end - start)

  return solver.Objective().Value(), allocation


def calculateEF1withmaxUSW(valuation_profile):
  solver = pywraplp.Solver.CreateSolver('SCIP')
  infinity = solver.infinity()
  M = 9999999
  x = {}
  for i in range(num_agents):
    for k in range(num_items):
      x[i,k] = solver.IntVar(0.0, infinity, 'x[%i,%i]' %(i,k))
      ## allocation can be either 0 or 1
      solver.Add(x[i,k] <= 1)

  ## get maximum item valued of each other's bundle
  y = {}
  for i in range(num_agents):
    for j in range(num_agents):
      y[i,j] = solver.NumVar(0.0, infinity, 'y[%i,%i]' %(i,j))

  b = {}
  for i in range(num_agents):
    for j in range(num_agents):
      for k in range(num_items):
        b[i,j,k] =  solver.IntVar(0.0, infinity, 'b[%i,%i,%i]' %(i,j,k))
        solver.Add(b[i,j,k] <= 1)


  print('Number of variables =', solver.NumVariables())
  print(solver)

  ## eack item should be only allocated once
  for k in range(num_items):
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) <=1)
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) >=1)
  
  ## Complete allocation 
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) <= num_items)
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) >= num_items)
  
  ## envyfreeness constraint
  for i in range(num_agents):
    for j in range(num_agents):
      # print('j', j)
      ## i shouldnot envy j 
      ## just do for goods rn
      #### chose y[i,j] as maximum valued item in jth bundle for agent i
      for k in range(num_items):
        solver.Add(valuation_profile[i,k]* x[j,k] <= y[i,j])
        solver.Add(valuation_profile[i,k]* x[j,k] + (1-b[i,j,k])*M >= y[i,j])
        
      solver.Add(solver.Sum(b[i,j,k] for k in range(num_items)) <=1)
      solver.Add(solver.Sum(b[i,j,k] for k in range(num_items)) >=1)
      solver.Add(solver.Sum(valuation_profile[i,k]* x[i,k] for k in range(num_items)) >= solver.Sum(valuation_profile[i,k]* x[j,k] for k in range(num_items) ) -y[i,j] )
      # j shouldnot envy i
      # solver.Add(solver.Sum(valuation_profile[j,k]* x[j,k] for k in range(num_items)) >= solver.Sum(valuation_profile[j,k]* x[i,k] for k in range(num_items) ))

      # solver.Add(solver.Sum(valuation_profile[i,k]* x[i,k] for k in range(num_items)) >= solver.Sum(valuation_profile[i,k]* x[j,k] for k in range(num_items) ) -y[j] )
      # ## j shouldnot envy i
      # solver.Add(solver.Sum(valuation_profile[j,k]* x[j,k] for k in range(num_items)) >= solver.Sum(valuation_profile[j,k]* x[i,k] for k in range(num_items) ))

  
  print('Number of constraints =', solver.NumConstraints())
  print(solver.constraints())
  ## Maximize the bundle
  solver.Maximize(solver.Sum(valuation_profile[i,k]*x[i,k] for i in range(num_agents) for k in range(num_items)))
  start = time.time()
  print("hello")
  status = solver.Solve()
  end = time.time()

  # print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

  if status == pywraplp.Solver.OPTIMAL:
      print('Solution:')
      print('Objective value EEF1 is =', solver.Objective().Value())
      for i in range(num_agents):
        print('______________________________',i,'______________________________')
        for k in range(num_items):
          print('x =', x[i,k].solution_value())
        print('________________________________________________')
      
      # for i in range(num_agents):
      #   print('______________________________',i,'______________________________')
      #   for j in range(num_agents):
      #     print('y =', y[i,j].solution_value())
      #   print('________________________________________________')

  else:
      print('The problem does not have an optimal solution.')
  print(end - start)

  return solver.Objective().Value()

def calculateProp(valuation_profile):
  proportionality = np.sum(valuation_profile, axis=1) / num_agents
  print('proportionality',proportionality)
  solver = pywraplp.Solver.CreateSolver('SCIP')
  infinity = solver.infinity()

  x = {}
  for i in range(num_agents):
    for k in range(num_items):
      x[i,k] =  x1 = solver.IntVar(0.0, infinity, 'x[%i,%i]' %(i,k))
      ## allocation can be either 0 or 1
      solver.Add(x[i,k] <= 1)

  y = solver.NumVar(0,infinity,'y')

  print('Number of variables =', solver.NumVariables())
  print(solver)

  ## eack item should be only allocated once
  for k in range(num_items):
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) <=1)
    solver.Add(solver.Sum(x[i,k] for i in range(num_agents)) >=1)
  
  ## Complete allocation 
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) <= num_items)
  solver.Add(solver.Sum(x[i,k] for i in range(num_agents) for k in range(num_items)) >= num_items)
  
  ## envyfreeness constraint
  for i in range(num_agents):
    ## proportional
    solver.Add(solver.Sum(valuation_profile[i,k]* x[i,k] for k in range(num_items)) >= proportionality[i])


  
  print('Number of constraints =', solver.NumConstraints())
  print(solver.constraints())
  ## Maximize the bundle
  solver.Maximize(1)
  status = solver.Solve()

  if status == pywraplp.Solver.OPTIMAL:
      print('Solution:')
      print('Objective value MMS Share is =', solver.Objective().Value())
      for i in range(num_agents):
        print('______________________________',i,'______________________________')
        for k in range(num_items):
          print('x =', x[i,k].solution_value())
        print('________________________________________________')
      return True
  else:
      print('The problem does not have an optimal solution.')
      return False
  # return solver.Objective().Value()





cpu


In [ ]:
from torch import nn, optim

def isSampleEFFree(allocation_profile,valuation_profile):
    n_agents = allocation_profile.shape[0]
    n_items = allocation_profile.shape[1]

    sumenvy = 0

    ## V_i A_j for all j
    multiply_vi_aj =  valuation_profile[:,None,:] * allocation_profile 

    ## V_i A_i
    multiply_vi_ai = valuation_profile * allocation_profile

    ## social welfare calculation for agent i corresponding to its own bundle and other
    ## agent's bundle
    sw_i_j = torch.sum(multiply_vi_aj , dim=2)
    sw_i = torch.sum(multiply_vi_ai , dim=1)
    print(sw_i_j.shape)
    print(sw_i.shape)

    ### Envy is calculated as the minimum envy by either removing item from other agent
    ### or by removing item from agent i bundle 
    envy,_ = torch.min( sw_i_j -  sw_i[:,None] , 0 )
    print(envy)
    print(envy.shape)
    ## filter out and only consider the positive envy value
    # relufn = nn.ReLU()
    # envy =  relufn(envy)
    envy[envy < 0] = 0
    sumenvy += torch.sum(envy)

    if (sumenvy == 0):
        return True
    else:
        return False


def findMUW(x):
    maxswallocation, indices = torch.max(x , dim=0)
    n_items = indices.shape[0]
    allocation_ = torch.zeros(x.shape).to(device)
    for n_item in range(n_items):
        agentid = indices[n_item]
        allocation_[agentid][n_item] =1 
    return allocation_

def calculatesw(allocation,valuation):
  return torch.sum(allocation * valuation)

In [ ]:
# num_agents = 3
# num_items = 12
# valuation = np.array([[380,349,330,320,310,273,219,210,130,120,109,100],[380,349,330,320,310,273,220,209,130,119,110,100],[380,350,329,320,310,273,219,210,129,120,110,100]])
# calculateMMSvalue(valuation_profile[0])
# mmsvalue =[950,950,950]
# findMMSAllocation(valuation_profile)

In [ ]:
num_agents = 3
num_items = 13

import numpy as np
# concentration = np.ones(num_items) * 10
# valuation = np.random.dirichlet(concentration, num_agents)
valuation1 = 400*np.random.normal(2, 0.0001, (num_agents,2))
valuation2 = 200*np.random.normal(2, 0.0001, (num_agents,2))
valuation3 = 10*np.random.normal(2, 0.0001, (num_agents,2))
valuation4 = 300*np.random.normal(2, 0.0001, (num_agents,2))
valuation5 = 350*np.random.normal(2, 0.0001, (num_agents,2))
valuation6 = 50*np.random.normal(2, 0.0001, (num_agents,2))

valuation = np.concatenate((valuation1, valuation2,valuation3,valuation4,valuation5,valuation6), axis=1)
# valuation = np.random.rand(num_agents,num_items)
# valuation = np.array([[5,5,5,5],[5,5,5,5]])
valuation = np.round(valuation, 1)
valuation = valuation*10
print(valuation)
print(valuation.shape)

[[8000. 8000. 4000. 4000.  200.  200. 6000. 6000. 7000. 7000. 1000. 1000.]
 [8000. 8000. 4000. 4000.  200.  200. 6000. 6000. 6999. 7000. 1000. 1000.]
 [8000. 8000. 4000. 4000.  200.  200. 6000. 6001. 7000. 7000. 1000. 1000.]]
(3, 12)


In [ ]:
# valuation = np.array([[7999, 7500, 4000, 4000,  200,  199, 6000, 6000, 7000, 7000,1000, 1001],
#  [8000, 7499, 4000, 3999,  201,  200, 6000, 6000, 7000, 7001, 999, 1000],
#  [7999, 7500, 4000, 4001,  199,  201, 6000, 6000, 7000, 6999, 1001, 999]])
import numpy as np

num_agents = 10
num_items = 100

# valuation = np.random.rand(num_agents,num_items)
# valuation = np.random.lognormal(mean=0.5, sigma=1.0, size=(num_agents,num_items))
# valuation = np.random.exponential(scale=1.0, size=(num_agents,num_items))

mu, sigma = 0.5, 0.1 # mean and standard deviation
valuation = np.random.normal(mu, sigma, size=(num_agents,num_items))

# valuation = np.array([
#                       [3801,3490,3300,3200,3100,2730,2190,2100,1300,1200,1090,1001],
#                       [3801,3490,3300,3200,3100,2730,2200,2090,1300,1190,1100,1001],
#                       [3801,3500,3290,3200,3100,2730,2190,2100,1290,1200,1101,1000]])

# valuation = np.array([
#                       [3805,3490,3300,3200,3100,2730,2190,2100,1300,1200,1090,1001],
#                       [3805,3490,3300,3200,3100,2730,2200,2090,1300,1190,1100,1001],
#                       [3805,3500,3290,3200,3100,2730,2190,2100,1290,1200,1101,1000]])


# valuation = np.array([
#                       [3490,3300,3200,3100,2730,2190,2100,1300,1200,1090,1000,1],
#                       [3490,3300,3200,3100,2730,2200,2090,1300,1190,1100,1000,1],
#                       [3500,3290,3200,3100,2730,2190,2100,1290,1200,1100,1000,1]])



# [[ 8763.  6282.  9059.  7854.  6380. 10607.  8360.  8938.  7245.  9379.
#    7624.  9510.]
#  [ 7687.  7993.  8349.  7344.  8678.  8409.  8544.  7518.  6714.  9697.
#    9130.  9939.]
#  [ 7279.  9281.  8122. 10018.  8301.  8152.  7581.  6798.  9907.  9059.
#    6512.  8990.]]


# [[ 75.  84.  84.  69.  87.  86.  83.  77.  88.  89.  91.  87.]
#  [ 87.  93.  68.  73.  87.  86.  77.  86.  89.  86.  77.  91.]
#  [ 74.  98. 100.  76.  76.  97.  74.  81.  94.  82.  76.  72.]]
# print(valuation[0])
# calculateMMSvalue(valuation[2])
# mmsvalue =[33309,33319,33269]

# mmsvalue =[333,333,331]
# findMMSAllocation(valuation,[])
# calculateEF(valuation)
# calculateProp(valuation)
# num_agents = 2
# num_items = 4
# valuation = np.array([[2,4,3,6],[3,5,6,7]])
# valuation = torch.from_numpy(valuation).float().to(device)
# print(valuation.device)
import time
# calculateEF1withmaxUSW(valuation)
value , allocation = calculateEFwihUSW(valuation)
# print(allocation)
valuation = torch.from_numpy(valuation)
allocation = torch.from_numpy(allocation)
print(allocation.shape)
print(isSampleEFFree(allocation,valuation))

print(calculatesw(findMUW(valuation),valuation))
# calculateEFwihUSW

Number of variables = 1001
<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x7f1e9ca9d6f0> >
Number of constraints = 1292
[<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f1e9b8d8630> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f1e9b8d8660> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f1e9b8d86c0> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f1e9b8d8720> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f1e9b8d8780> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f1e9b8d87e0> >, <ortools.linea

1) 11.519833326339722
Solution:
Objective value EEF1 is = 83.90281966404181

298.9722156524658
Solution:
Objective value EEF1 is = 86.1997239338344



In [ ]:
# Create the mip solver with the SCIP backend.
# solver = pywraplp.Solver.CreateSolver('SCIP')
# infinity = solver.infinity()
# x and y are integer non-negative variables.
# x1 = solver.IntVar(0.0, infinity, 'x1')
# x2 = solver.IntVar(0.0, infinity, 'x2')
# x3 = solver.IntVar(0.0, infinity, 'x3')
# x4 = solver.IntVar(0.0, infinity, 'x4')
# x5 = solver.IntVar(0.0, infinity, 'x5')
# x6 = solver.IntVar(0.0, infinity, 'x6')
# x7 = solver.IntVar(0.0, infinity, 'x7')
# x8 = solver.IntVar(0.0, infinity, 'x8')
# x9 = solver.IntVar(0.0, infinity, 'x9')
# x10 = solver.IntVar(0.0, infinity, 'x10')
# x11 = solver.IntVar(0.0, infinity, 'x11')
# x12 = solver.IntVar(0.0, infinity, 'x12')


# solver.Add(x1 <= 1)
# solver.Add(x2 <= 1)
# solver.Add(x3 <= 1)
# solver.Add(x4 <= 1)
# solver.Add(x5 <= 1)
# solver.Add(x6 <= 1)
# solver.Add(x7 <= 1)
# solver.Add(x8 <= 1)
# solver.Add(x9 <= 1)
# solver.Add(x10 <= 1)
# solver.Add(x11 <= 1)
# solver.Add(x12 <= 1)

# # 1,2,7
# solver.Add(3800*x1 + 3490 * x2 + 3300*x3 + 3200*x4 + 3100*x5 + 2730*x6 + 2190*x7 + 2100*x8+ 1300*x9 + 1200*x10+ 1090*x11 + 1000*x12 <= 9500)
# solver.Add(3800*x1 + 3490 * x2 + 3300*x3 + 3200*x4 + 3100*x5 + 2730*x6 + 2190*x7 + 2100*x8+ 1300*x9 + 1200*x10+ 1090*x11 + 1000*x12 >= 9500)

# #Agent 2
# #
# # solver.Add(0*x1 + 349 * x2 + 330*x3 + 320*x4 + 310*x5 + 273*x6 + 0*x7 + 209*x8+ 0*x9 + 0*x10+ 110*x11 + 0*x12 <= 949)
# # solver.Add(0*x1 + 349 * x2 + 330*x3 + 320*x4 + 310*x5 + 273*x6 + 0*x7 + 209*x8+ 0*x9 + 0*x10+ 110*x11 + 0*x12 >= 949)

# #Agent 3
# # 1,7,9,10,12
# solver.Add(380*x1 + 350 * x2 + 329*x3 + 320*x4 + 310*x5 + 273*x6 + 219*x7 + 210*x8+ 129*x9 + 120*x10+ 110*x11 + 100*x12 <= 948)
# solver.Add(380*x1 + 350 * x2 + 329*x3 + 320*x4 + 310*x5 + 273*x6 + 219*x7 + 210*x8+ 129*x9 + 120*x10+ 110*x11 + 100*x12 >= 948)

# solver.Add(0*x1 + 349 * x2 + 330*x3 + 320*x4 + 310*x5 + 273*x6 + 219*x7 + 210*x8+ 130*x9 + 120*x10+ 109*x11 + 100*x12 <= 948)
# solver.Add(0*x1 + 349 * x2 + 330*x3 + 320*x4 + 310*x5 + 273*x6 + 219*x7 + 210*x8+ 130*x9 + 120*x10+ 109*x11 + 100*x12 >= 948)


# x <= 3.5.
# solver.Add(x <= 3.5)

# print('Number of constraints =', solver.NumConstraints())
# # Maximize x + 10 * y.
# solver.Maximize(1)
# status = solver.Solve()

# if status == pywraplp.Solver.OPTIMAL:
#     print('Solution:')
#     print('Objective value =', solver.Objective().Value())
#     print('x1 =', x1.solution_value())
#     print('x2 =', x2.solution_value())
#     print('x3 =', x3.solution_value())
#     print('x4 =', x4.solution_value())
#     print('x5 =', x5.solution_value())
#     print('x6 =', x6.solution_value())
#     print('x7 =', x7.solution_value())
#     print('x8 =', x8.solution_value())
#     print('x9 =', x9.solution_value())
#     print('x10 =', x10.solution_value())
#     print('x11 =', x11.solution_value())
#     print('x12 =', x12.solution_value())
# else:
#     print('The problem does not have an optimal solution.')